In [1]:
#!cp simplify.txt cement_pipeline_pdal.json

In [1]:
! cat cement_pipeline_pdal.json

{
    "pipeline": [
        {
"bounds": "([-10425724.019241972, -10422962.731595168], [5162298.915, 5164878.665])",
"filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_NorthCentral_2_2020/ept.json",
"type": "readers.ept",
"tag": "readdata"
        },
        {
            "out_srs": "EPSG:26915",
            "tag": "reprojectUTM",
            "type": "filters.reprojection"
        },
        {
            "filename": "cement.laz",
            "inputs": [ "reprojectUTM" ],
            "tag": "writerslas",
            "type": "writers.las"
        }
    ]
}


In [2]:
#! aws s3 cp s3://usgs-lidar-public/IA_NorthCentral_2_2020/ept.json - | grep horizontal

https://epsg.io/3857

In [3]:
#! wget https://raw.githubusercontent.com/tonybutzer/composite/41771c607f8f073b042b399080cd0f0bcc94a46a/00-notebooks/05-04-2021-FIRE/fakelib/fm_map.py

In [2]:
from fm_map import Fmap

In [3]:
! ls *.geojson

cement.geojson


In [4]:
m = Fmap('blue')

creating Fmap class


In [5]:
m.sat_geojson('cement.geojson')

In [6]:
import geopandas as gpd

df = gpd.read_file('cement.geojson')

In [7]:
df

,geometry
0,"POLYGON ((-93.65587 42.00881, -93.63107 42.008..."


In [8]:
dfn = df.to_crs(crs='EPSG:3857')

In [9]:
dfn

,geometry
0,"POLYGON ((-10425724.019 5162298.915, -10422962..."


In [10]:
df

,geometry
0,"POLYGON ((-93.65587 42.00881, -93.63107 42.008..."


In [11]:
dfn.iloc[0].geometry.boundary.bounds

(-10425724.019241972,
 5162298.915410344,
 -10422962.731595168,
 5164878.665114964)

In [12]:
dir (dfn.iloc[0].geometry.boundary)

['__and__',
 '__array_interface__',
 '__bool__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__geom__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__p__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '__xor__',
 '_array_interface',
 '_array_interface_base',
 '_crs',
 '_ctypes',
 '_ctypes_data',
 '_empty',
 '_geom',
 '_get_coords',
 '_is_empty',
 '_lgeos',
 '_ndim',
 '_other_owned',
 '_repr_svg_',
 '_set_coords',
 '_set_geom',
 'almost_equals',
 'area',
 'array_interface',
 'array_interface_base',
 'boundary',
 'bounds',
 'buffer',
 'centroid',
 'contains',
 'convex_hull',
 'coords',
 'covered_by',
 'covers',
 'crosses',
 'ctypes',
 'difference',
 'disjoint',


# now edit cement_pdal.json with bounds
vi 

In [13]:
! pdal pipeline cement_pipeline_pdal.json --debug

(PDAL Debug) Debugging...
(pdal pipeline readers.ept Debug) Query bounds: ([-10425724.01924197, -10422962.73159517], [5162298.915, 5164878.665], [-1.797693134862316e+308, 1.797693134862316e+308])
Threads: 15
(pdal pipeline Debug) Executing pipeline in stream mode.
(pdal pipeline writers.las Debug) Wrote 12661669 points to the LAS file


In [14]:
!ls -lh cement.laz

-rw-rw-r-- 1 ec2-user ec2-user 59M May 13 19:39 cement.laz


In [15]:
! mv cement.laz  ~/ia

In [16]:
! docker run -it -v /home/ec2-user/ia:/entwine connormanning/entwine build -i /entwine/cement.laz -o /entwine/CEMENT

Awakening existing build.
Loading cement.json from /entwine/CEMENT/ept-sources/
Dimensions: [
        X:int32, Y:int32, Z:int32, Intensity:uint16, ReturnNumber:uint8,
        NumberOfReturns:uint8, ScanDirectionFlag:uint8, EdgeOfFlightLine:uint8,
        Classification:uint8, ScanAngleRank:float32, UserData:uint8,
        PointSourceId:uint16, GpsTime:float64, Red:uint16, Green:uint16, Blue:uint16
]
Points: 12,661,669
Bounds: [(445689, 4650946, 231), (447758, 4652874, 431)]
Scale: 0.01
SRS: EPSG:26915

Joining
Saving
Wrote 0 points.


In [ ]:
# use connor manning http server on port 80

! docker run -it -v ~/ia:/var/www -p 8080:8080 connormanning/http-server

Starting up http-server, serving /var/www
Available on:
  http://127.0.0.1:8080
  http://172.17.0.2:8080
Hit CTRL-C to stop the server
[Fri May 13 2022 19:43:18 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept.json" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:19 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-hierarchy/0-0-0-0.json" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:19 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/0-0-0-0.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:25 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/1-1-0-0.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:25 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/1-0-0-0.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x

[Fri May 13 2022 19:43:35 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-3-0-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:36 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-5-3-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:36 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-4-2-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:36 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-3-1-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:36 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-6-4-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:36 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-dat

[Fri May 13 2022 19:43:43 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-0-4-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:43 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-3-7-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:43 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-1-6-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:43 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-2-7-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:44 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/3-0-5-3.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:43:44 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-dat

[Fri May 13 2022 19:44:18 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/4-1-6-7.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:44:18 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/4-6-7-7.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:44:18 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/4-0-5-7.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:44:19 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/4-2-7-7.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:44:32 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-data/4-6-2-7.laz" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"
[Fri May 13 2022 19:44:33 GMT+0000 (Coordinated Universal Time)] "GET /CEMENT/ept-dat

- https://potree.entwine.io/data/custom.html?r=http://localhost:8080/CEMENT/ept.json